#Dataset 1 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
file_1 = '/content/gdrive/MyDrive/summarisation_dataset/essg_dataframe.pkl'
data_frame = pd.read_pickle(file_1)
data_frame.head()

,story,summary,filename
0,"In the 17th century, Isaac Newton discovered t...",[[comparing newton's observation of prismatic ...,10.txt
1,﻿A sea is a large body of salt water that is s...,[[owing to the present state of continental dr...,100.txt
2,﻿Nazi Germany and the Third Reich (German: Dri...,[[nazi germany and the third reich (german: dr...,1.txt
3,A rainbow is a meteorological phenomenon that ...,"[[rainbows can be full circles; however, the a...",11.txt
4,"﻿Shiva meaning ""The Auspicious One"", also kno...","[[he is the parabrahman within shaivism, one o...",26.txt


In [ ]:
data_frame.to_csv('essg_dataframe.csv')

In [ ]:
file_path = '/content/gdrive/MyDrive/summarisation_dataset/tfidf_keywords_dataframe.pkl'
df_keywords = pd.read_pickle(file_path)
df_keywords.head()  # original df

,story,top_keywords,summmary,filename
0,"In the 17th century, Isaac Newton discovered t...","[color, spectrum, blue, indigo, newton, light,...",[[comparing newton's observation of prismatic ...,10.txt
1,﻿A sea is a large body of salt water that is s...,"[sea, ocean, water, land, produc, cycl, whale,...",[[owing to the present state of continental dr...,100.txt
2,﻿Nazi Germany and the Third Reich (German: Dri...,"[hitler, germani, reich, nazi, german, power, ...",[[nazi germany and the third reich (german: dr...,1.txt
3,A rainbow is a meteorological phenomenon that ...,"[rainbow, arc, droplet, refract, sky, inner, l...","[[rainbows can be full circles; however, the a...",11.txt
4,"﻿Shiva meaning ""The Auspicious One"", also kno...","[shiva, god, hinduism, benevol, worship, depic...","[[he is the parabrahman within shaivism, one o...",26.txt


In [ ]:
file_path = '/content/gdrive/MyDrive/summarisation_dataset/tfidf_keywords_dataframe.pkl'
df1 = pd.read_pickle(file_path)  # use for data cleaning part 

#Data Cleaning


In [ ]:
# Data Cleaning

# separate sentences using spacy -> primitive cleaning -> stop word removal -> lemmatization -> stemming
# update story column with a list of sentences 

In [ ]:
# separate sentences using spacy 

import spacy
nlp = spacy.load('en_core_web_sm')
import numpy as np
import pandas as pd

def separate_sentences(paragraph):
  sent_list = []
  paragraph = nlp(paragraph)
  for sents in paragraph.sents:
    sent_list.append(sents.text)
  return sent_list

# separate sentences using regex

import re

def sent_separation(par):
  par = par.replace('\n'," ").replace('\r', '')
  #par = par.replace('(',"").replace(')','')
  par = par.replace('"',' ')
  #m = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', par)
  #(?<![A-Z][a-z]\.)(?<=\.|\?) +(?=[A-Z0-9])
  m = re.split('(?<![A-Z][a-z]\.)(?<=\.|\?) +(?=[A-Z0-9])', par)
  sentences=[]
  for i in m:
    sentences.append(i)
  return sentences


In [ ]:
df1['story'] = df1['story'].apply(sent_separation) 

In [ ]:
!pip install contractions

In [ ]:
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""" 

In [ ]:
# primitive cleaning 

import unicodedata
import re
import contractions

TAG_RE = re.compile(r'<[^>]+>')  # remove html tags

def remove_tags(text):
  return TAG_RE.sub('',text)

def expand_contractions(text):
    return contractions.fix(text)  # I'll -> I  will 

def primitive_cleaning_sentence(sen):
  sentence = remove_tags(sen)
  sentence = sentence.translate(sentence.maketrans("\n\t\r", "   "))

  # sentence = sentence.replace(".","")  
  # not required, done already

  sentence = expand_contractions(sentence)
  sentence = re.sub('[^a-zA-Z]', ' ', sentence) # 17th -> after removal becomes 'th' ?
  sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
  sentence = re.sub(r'\s+', ' ', sentence)  #  one or more whitespace characters
  sentence = sentence.lower()

  # Removing punctuation from the ssentence
  sentence = "".join([c for c in sentence if c not in PUNCTUATION])  

  return sentence

def primitive_clean_row(sent_list):
  for i in range(len(sent_list)):
    sent_list[i] = primitive_cleaning_sentence(sent_list[i])
  return sent_list

In [ ]:
df1['story'] = df1['story'].apply(primitive_clean_row)

In [ ]:
# Stop word removal

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def stop_word_removal_sent(text):
  word_tokens = word_tokenize(text)
  new_sent = ""

  for w in word_tokens:
    if w.lower() not in stop_words:
      if new_sent == "":
        new_sent += w
      else:
        new_sent += " " + w 

  return new_sent  

def stop_word_removal_row(sent_list):
  for i in range(len(sent_list)):
    sent_list[i] = stop_word_removal_sent(sent_list[i])
  return sent_list


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df1['story'] = df1['story'].apply(stop_word_removal_row)

In [ ]:
# Lemmatization and Stemming

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def lemmatize_a_sent(text):
  lemmatizer = WordNetLemmatizer()
  word_list = nltk.word_tokenize(text)  # Tokenize: Split the sentence into words
  text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])  # Lemmatize list of words and join
  return text 

def lemmatize_a_row(sent_list):
  for i in range(len(sent_list)):
    sent_list[i] = lemmatize_a_sent(sent_list[i])
  return sent_list

def stem_a_sent(text):
  ps = PorterStemmer()
  word_list = nltk.word_tokenize(text)  # Tokenize: Split the sentence into words
  text = ' '.join([ps.stem(w) for w in word_list])  # stem list of words and join
  return text

def stem_a_row(sent_list):
  for i in range(len(sent_list)):
    sent_list[i] = stem_a_sent(sent_list[i])
  return sent_list

In [ ]:
df1['story'] = df1['story'].apply(lemmatize_a_row)

In [ ]:
df1['story'] = df1['story'].apply(stem_a_row)

stemming  <br>

otherwise -> otherwise,
edge -> edg

# Hypergraph Creation

In [ ]:
file_path = '/content/gdrive/MyDrive/summarisation_dataset/tfidf_keywords_dataframe.pkl'
df2 = pd.read_pickle(file_path)  # use for hypergraph part 
# 'story' column of df2 will have the hypergraph dictionaries for all the stories

In [ ]:
# find sentences that contain a particular keyword -> convert into a dictionary 
# will have to check this for n-grams ?

# hypergraph -> dictionay -> 
# key -> keyword 
# value -> list of indices sentences containing that keyword 

import re

# using regex

# def create_hypergraph_row(sent_list, keyword_array):
#   txt = ' . '.join(str(x) for x in sent_list)  # one paragraph
#   hypergraph_dict = dict()  # hypergraph for one document 
#   for word in keyword_array:
#       exprn = r'([^.]*'+ word +'[^.]*)'          # required sentence 
#       hypergraph_dict[word] = re.findall(exprn, txt)  # find all sentences in the paragraph containing this keyword.
#   return hypergraph_dict

# simply check if the word is preseent in the sentence

def create_hypergraph_row(sent_list, keyword_array):
  hypergraph_dict = dict()  # hypergraph for one document
  for i in range(len(keyword_array)):
    hypergraph_dict[keyword_array[i]] = []  # initialize a list of sentences
    for j in range(len(sent_list)):
      if keyword_array[i] in sent_list[j]:
        hypergraph_dict[keyword_array[i]].append(j)  # add the index of the sentence
  return hypergraph_dict


In [ ]:
df2['story'] = df1.apply(lambda x: create_hypergraph_row(x.story, x.top_keywords), axis = 1)

#Domination


In [ ]:
file_path = '/content/gdrive/MyDrive/summarisation_dataset/tfidf_keywords_dataframe.pkl'
df3 = pd.read_pickle(file_path)  # use for hypergraph part 
# 'story' column of df2 will have the hypergraph dictionaries for all the stories

In [ ]:
def clean_sent_for_summary(sent):
  sentence = remove_tags(sent)  # remove html tags
  sentence = sentence.translate(sentence.maketrans("\n\t\r", "   "))  
  sentence = expand_contractions(sentence)
  sentence = re.sub(r'\s+', ' ', sentence)  #  one or more whitespace characters
  return sentence

def clean_row_for_summary(sent_list):
  for i in range(len(sent_list)):
    sent_list[i] = clean_sent_for_summary(sent_list[i])
  return sent_list

In [ ]:
file_path = '/content/gdrive/MyDrive/summarisation_dataset/tfidf_keywords_dataframe.pkl'
df4 = pd.read_pickle(file_path)  # only for separating sentences from the original paragraph
# sentences from this dataframe will be put in the hypothesized summary 
# story column will be a list of separated sentences

df4['story'] = df4['story'].apply(sent_separation)  # function updated
df4['story'] = df4['story'].apply(clean_row_for_summary)

In [ ]:
# Newton\'s -> text cleaning -> handling this '\' ? 

In [ ]:
!pip install hypernetx
!pip install celluloid
!pip install igraph

In [ ]:
import networkx as nx
import hypernetx as hnx

In [ ]:
# function to find the set in the list of dominating sets with maximum number of elements
# return the first list with maximum number of elements

def find_list_with_max_elements(dom_list):
  no_of_elements = []  # find number of elements in every set 
  for i in range(len(dom_list)):
    no_of_elements.append(len(dom_list[i]))  # length of every dominating set
  max_ele = max(no_of_elements)

  for i in range(len(dom_list)):
    if len(dom_list[i]) == max_ele:
      return i 


In [ ]:
# list of dominating set (original sentences) go in the 'story' column
# original sentence in the story should be added in the summary 

def domination_row(keyword_sent_dict):
  # finding neighbors
  h = hnx.Hypergraph(keyword_sent_dict)
  li = []
  for i in list(h.nodes):
      li.append(h.neighbors(i))

  # Mapping nodes to numbers
  mapp = {}
  c = 0
  for i in list(h.nodes):
      mapp[i] = c
      c = c + 1

  # GFG code
  # s = []
  ver = len(li)
  # visit = np.zeros(ver)

  deg = {}
  for i in range(len(li)):
      deg[i] = len(li[i])

  deg = dict(sorted(deg.items(), key = lambda item: item[1], reverse = True))

  dom_li = []

  # for i in deg.keys():
  #     if(visit[i] == 0):
  #         s.append(i)
  #         visit[i] = 1
  #         for j in range(len(li[i])):
  #             if(visit[mapp[li[i][j]]] == 0):
  #                 visit[mapp[li[i][j]]] = 1

  for di in range(ver):
    visit = np.zeros(ver)
    s = []
    
    k = list(deg.keys())[0]
    v = list(deg.values())[0]
    for i in deg.keys():
        if(visit[i] == 0):
            s.append(i)
            visit[i]=1
            for j in range(len(li[i])):
                if(visit[mapp[li[i][j]]]==0):
                    visit[mapp[li[i][j]]]=1
    deg.pop(list(deg.keys())[0])
    deg[k] = v
    dom_li.append(s)

  return sorted(dom_li[find_list_with_max_elements(dom_li)])  # returns the dominating set with the maximum elements

In [ ]:
df3['story'] = df2['story'].apply(domination_row)

In [ ]:
df3['story']

0                    [4, 14, 16]
1                [0, 18, 20, 22]
2                 [4, 9, 11, 12]
3                         [0, 6]
4                  [5, 8, 9, 10]
                 ...            
95               [5, 12, 16, 17]
96                   [0, 15, 17]
97           [1, 13, 15, 16, 17]
98                    [2, 7, 13]
99    [7, 9, 13, 14, 17, 18, 20]
Name: story, Length: 100, dtype: object

In [ ]:
df3['story'].str.len().value_counts()

2    32
4    26
3    23
5     8
1     6
6     3
7     2
Name: story, dtype: int64

In [ ]:
file_path = '/content/gdrive/MyDrive/summarisation_dataset/tfidf_keywords_dataframe.pkl'
df5 = pd.read_pickle(file_path)  # final df containing both hypothesised and final summaries  

In [ ]:
# convert the story column into actual summaries 
# wrap this into a function ?

hypothesized_summaries = []  # making a new column for the df
for i in range(len(df3)):
  # for a row 
  hypothesized_summary = []  # list of sentences 
  for j in df3['story'][i]:
    hypothesized_summary.append(df4['story'][i][j])  # jth element in the list of sentences 
  hypothesized_summaries.append(hypothesized_summary)

df5['hypothesized summary'] = hypothesized_summaries

In [ ]:
df5['story'] = df5['story'].apply(clean_sent_for_summary)  # sentence cleaning will also work for the entire paragraph

In [ ]:
df5.to_pickle('df_hypothesized_and_ref_summaries.pkl')

# ROUGE

In [ ]:
!pip install rouge

In [ ]:
df5.head()

,story,top_keywords,summmary,filename,hypothesized summary
0,"In the 17th century, Isaac Newton discovered t...","[color, spectrum, blue, indigo, newton, light,...",[[comparing newton's observation of prismatic ...,10.txt,Young was the first to measure the wavelengths...
1,﻿A sea is a large body of salt water that is s...,"[sea, ocean, water, land, produc, cycl, whale,...",[[owing to the present state of continental dr...,100.txt,"However, population growth, industrialization,..."
2,﻿Nazi Germany and the Third Reich (German: Dri...,"[hitler, germani, reich, nazi, german, power, ...",[[nazi germany and the third reich (german: dr...,1.txt,"Extensive public works were undertaken, includ..."
3,A rainbow is a meteorological phenomenon that ...,"[rainbow, arc, droplet, refract, sky, inner, l...","[[rainbows can be full circles; however, the a...",11.txt,Rainbows caused by sunlight always appear in t...
4,"﻿Shiva meaning ""The Auspicious One"", also know...","[shiva, god, hinduism, benevol, worship, depic...","[[he is the parabrahman within shaivism, one o...",26.txt,Shiva is also regarded as the patron god of yo...


In [ ]:
df5['hypothesized summary'][1]

['More broadly, "the sea" is the interconnected system of Earth\'s salty, oceanic waters—considered as one global ocean or as several principal oceanic divisions.',
 'Atmospheric carbon dioxide is being absorbed in increasing amounts, lowering its pH in a process known as ocean acidification.',
 'It is the scene of leisure activities including swimming, diving, surfing, and sailing.',
 'However, population growth, industrialization, and intensive farming have all contributed to present-day marine pollution.']

In [ ]:
type(df5['summmary'][1][0])

str

In [ ]:
df5['summmary'][1]

['[owing to the present state of continental drift, the northern hemisphere is now fairly equally divided between land and sea (a ratio of about 2:3) but the south is overwhelmingly oceanic (1:4.7). salinity in the open ocean is generally in a narrow band around 3.5% by mass, although this can vary in more landlocked waters, near the mouths of large rivers, or at great depths., although the sea has been travelled and explored since prehistory, the modern scientific study of the sea—oceanography—dates broadly to the british challenger expedition of the 1870s. the sea is conventionally divided into up to five large oceanic sections—including the iho\'s four named oceans (the atlantic, pacific, indian, and arctic) and the southern ocean; smaller, second-order sections, such as the mediterranean, are known as seas., oceanography has established that not all life is restricted to the sunlit surface waters: even under enormous depths and pressures, nutrients streaming from hydrothermal vents

In [ ]:
def clean_ref_sum(sum_list):
  

In [ ]:
from rouge import Rouge

rouge = Rouge()

model_out = []
reference = ["nazi germany and the third reich (german: drittes reich) are common english names for the period of history in germany from 1933 to 1945, when it was a dictatorship under the control of adolf hitler and the nazi party (nsdap).under hitler's rule, germany was transformed into a fascist totalitarian state which controlled nearly all aspects of life."]

rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'f': 0.08450703769093457, 'p': 0.12, 'r': 0.06521739130434782},
 'rouge-2': {'f': 0.02352940726920501,
  'p': 0.034482758620689655,
  'r': 0.017857142857142856},
 'rouge-l': {'f': 0.08450703769093457, 'p': 0.12, 'r': 0.06521739130434782}}

"f" stands for f1_score, "p" stands for precision, "r" stands for recall.